In [ ]:
#Importing pandas,numpy, and matplotlib 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer


#Forcing pandas to display all columns in our dataset 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

#This code allows you to view everything that you want to view 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
#Importing Training and Testing data. Target variable was included in training data spreadsheet via v-lookup in excel
training_data = pd.read_csv(r'Training.csv')
testing_data = pd.read_csv(r'Testing.csv')
submission_file = pd.read_csv(r'submission.csv')

# Preliminary Data Exploration 

In [ ]:
#Quick glimpse at the start and end of the data 
training_data.head()
training_data.tail()

In [ ]:
#training data has 59,400 records and 41 columns 
training_data.shape
testing_data.shape

#most of these numeric fields don't tell you much, they are mainly columns that will need to be transformed into factors
#to allow us to model correctly
training_data.describe().transpose()

In [ ]:
#checking our training data to identify which features contain missing values 
#missing values in the following features: funder, installer, subvillage, public meeting, scheme_management, scheme_name, 
# and permit. The largest number of missing values occurs in scheme_name (28,166) 
training_data.isnull().sum()

In [ ]:
for col in list(training_data):
    print(col)
    print(training_data[col].unique())

In [ ]:
#Identifying the number of unique values for each feature. There are a lot of features with a lot of unique values. We will
#need to identify those 'rare values' and combine them into an 'other' category
unique_values = training_data.nunique()
print(unique_values)

In [ ]:
#Idenitfying the levels within each feature to assist with data cleaning and rare category combination 
for col in training_data.select_dtypes(include=['category','object','bool']).columns:
    print(col)
    print(training_data[col].value_counts())
    print('\n')

# Data Cleaning

This section of code goes through various columns fixing spelling and grammar to ensure instances are grouped together correctly. This section also encodes any instances that are 'none' and 'unknown' to NaN to allow for encoding missing values later on in the script

In [ ]:
#Converting Target Variable to Binary

training_data.loc[(training_data['Status_group'] == 'functional'), 'Status_group'] = 0
training_data.loc[(training_data['Status_group'] == 'functional needs repair'), 'Status_group'] = 1
training_data.loc[(training_data['Status_group'] == 'non functional'), 'Status_group'] = 2

training_data['Status_group'].head()
training_data['Status_group'] = training_data['Status_group'].astype('int64')
#training_data.info()

In [ ]:
#Creating a new dataframe to perform data cleaning
training_data_2 = training_data.copy()
testing_data_2 = testing_data.copy()

In [ ]:
#Making funder features consistent in training data
training_data_2.loc[(training_data_2['funder'] == 'Rc Cathoric') | (training_data_2['funder'] == 'Roman Cathoric') |
                    (training_data_2['funder'] == 'Roman')|(training_data_2['funder'] == 'Roman Cathoic')|(training_data_2['funder'] == 'Rc')
                    |(training_data_2['funder'] == 'Rc Ch')|(training_data_2['funder'] == 'Rc Churc')
                    |(training_data_2['funder'] == 'Romam Catholic')|(training_data_2['funder'] == 'Roman Catholic')
                    |(training_data_2['funder'] == 'Roman Ca')|(training_data_2['funder'] == 'Roman Cathoric Church')
                    |(training_data_2['funder'] == 'Roman Cathoric-kilomeni')|(training_data_2['funder'] == 'Roman Cathoric Same')
                    |(training_data_2['funder'] == 'Roman Cathoric-same')|(training_data_2['funder'] == 'Roman Church'), 'funder'] = 'Rc Church'


training_data_2.loc[(training_data_2['funder'] == 'Germany Republi') | (training_data_2['funder'] == 'Germany'), 'funder'] = 'Germany Republic'

training_data_2.loc[(training_data_2['funder'] == 'Sweden') | (training_data_2['funder'] == 'Sweeden')
                    | (training_data_2['funder'] == 'Swidish'), 'funder'] = 'Swedish'


training_data_2.loc[(training_data_2['funder'] == 'Ces(gmbh)'), 'funder'] = 'Ces (gmbh)'

training_data_2.loc[(training_data_2['funder'] == 'Solidame') | (training_data_2['funder'] == 'Soliderm'), 'funder'] = 'Solidarm'

training_data_2.loc[(training_data_2['funder'] == 'Is'), 'funder'] = 'Isf'

training_data_2.loc[(training_data_2['funder'] == 'Kilwater'), 'funder'] = 'Killiwater'

training_data_2.loc[(training_data_2['funder'] == 'Magadini Makiwaru Water'), 'funder'] = 'Magadini-makiwaru Water'

training_data_2.loc[(training_data_2['funder'] == 'Lawate Fuka Water Suppl'), 'funder'] = 'Lawatefuka Water Supply'

training_data_2.loc[(training_data_2['funder'] == 'Jika')| (training_data_2['funder'] == 'Jeica')
                    | (training_data_2['funder'] == 'Jica'), 'funder'] = 'Jaica'

training_data_2.loc[(training_data_2['funder'] == 'Plan Int')| (training_data_2['funder'] == 'Plan Internatio'), 'funder'] = 'Plan International'

training_data_2.loc[(training_data_2['funder'] == 'Dwssp'), 'funder'] = 'Dwsp'

training_data_2.loc[(training_data_2['funder'] == 'Fin Water')| (training_data_2['funder'] == 'Finn Water')
                    | (training_data_2['funder'] == 'Finw')| (training_data_2['funder'] == 'Finwater')
                    | (training_data_2['funder'] == 'Fiwater'), 'funder'] = 'Fini Water'

training_data_2.loc[(training_data_2['funder'] == 'Hesaw')| (training_data_2['funder'] == 'Hesawa')
                    | (training_data_2['funder'] == 'Hesawwa')| (training_data_2['funder'] == 'Hesawza')
                    | (training_data_2['funder'] == 'Hesawza') | (training_data_2['funder'] == 'Hesswa')
                    | (training_data_2['funder'] == 'Hewasa')| (training_data_2['funder'] == 'Hewawa')
                    | (training_data_2['funder'] == 'Hhesawa'), 'funder'] = 'Hesawa'

training_data_2.loc[(training_data_2['funder'] == 'Rural Drinking Water Supply')| (training_data_2['funder'] == 'Rural Water Department')
                    | (training_data_2['funder'] == 'Rural Water Supply')| (training_data_2['funder'] == 'Rural Water Supply And Sanita')
                    | (training_data_2['funder'] == 'Rural Water Supply And Sanitat') 
                    | (training_data_2['funder'] == 'Rural Water Supply and Sanitat'), 'funder'] = 'Rural Water And Sanitation'

training_data_2.loc[(training_data_2['funder'] == 'Quick')| (training_data_2['funder'] == 'Quick Win')
                    | (training_data_2['funder'] == 'Quick Win Project')| (training_data_2['funder'] == 'Quick Win Project/council')
                    | (training_data_2['funder'] == 'Quick Win/halmashauri') | (training_data_2['funder'] == 'Quick Wings')
                    | (training_data_2['funder'] == 'Quick Wins')| (training_data_2['funder'] == 'Quick Wins Scheme')
                    | (training_data_2['funder'] == 'Quicklw') | (training_data_2['funder'] == 'Quickwi')
                    | (training_data_2['funder'] == 'Quickwins')| (training_data_2['funder'] == 'Quik')
                    | (training_data_2['funder'] == 'Qwckwin')| (training_data_2['funder'] == 'Qwekwin')
                    | (training_data_2['funder'] == 'Qwick Win') | (training_data_2['funder'] == 'Qwickwin'), 'funder'] = 'Quwkwin'

training_data_2.loc[(training_data_2['funder'] == 'Priva')| (training_data_2['funder'] == 'Private')
                    | (training_data_2['funder'] == 'Private Individul')| (training_data_2['funder'] == 'Private Person'), 'funder'] = 'Private Individual'

training_data_2.loc[(training_data_2['funder'] == 'Nerthlands')| (training_data_2['funder'] == 'Nethalan')
                    | (training_data_2['funder'] == 'Nethalan')| (training_data_2['funder'] == 'Netherland')
                    | (training_data_2['funder'] == 'Netherla'), 'funder'] = 'Netherlands'

training_data_2.loc[(training_data_2['funder'] == 'Missi')| (training_data_2['funder'] == 'Missio')
                    | (training_data_2['funder'] == 'Missionaries')| (training_data_2['funder'] == 'Missionary')
               , 'funder'] = 'Mission'

training_data_2.loc[(training_data_2['funder'] == 'Unice')| (training_data_2['funder'] == 'Uniceg')
                    | (training_data_2['funder'] == 'Unicet')| (training_data_2['funder'] == 'Unicrf')
                    | (training_data_2['funder'] == 'Uniseg')
               , 'funder'] = 'Unicef'

training_data_2.loc[(training_data_2['funder'] == '0') , 'funder'] = np.nan




#print(training_data_2['funder'].value_counts())

In [ ]:
#Making funder features consistent in test set 
testing_data_2.loc[(testing_data_2['funder'] == 'Rc Cathoric') | (testing_data_2['funder'] == 'Roman Cathoric') |
                    (testing_data_2['funder'] == 'Roman')|(testing_data_2['funder'] == 'Roman Cathoic')|(testing_data_2['funder'] == 'Rc')
                    |(testing_data_2['funder'] == 'Rc Ch')|(testing_data_2['funder'] == 'Rc Churc')
                    |(testing_data_2['funder'] == 'Romam Catholic')|(testing_data_2['funder'] == 'Roman Catholic')
                    |(testing_data_2['funder'] == 'Roman Ca')|(testing_data_2['funder'] == 'Roman Cathoric Church')
                    |(testing_data_2['funder'] == 'Roman Cathoric-kilomeni')|(testing_data_2['funder'] == 'Roman Cathoric Same')
                    |(testing_data_2['funder'] == 'Roman Cathoric-same')|(testing_data_2['funder'] == 'Roman Church'), 'funder'] = 'Rc Church'


testing_data_2.loc[(testing_data_2['funder'] == 'Germany Republi') | (testing_data_2['funder'] == 'Germany'), 'funder'] = 'Germany Republic'

testing_data_2.loc[(testing_data_2['funder'] == 'Sweden') | (testing_data_2['funder'] == 'Sweeden')
                    | (testing_data_2['funder'] == 'Swidish'), 'funder'] = 'Swedish'


testing_data_2.loc[(testing_data_2['funder'] == 'Ces(gmbh)'), 'funder'] = 'Ces (gmbh)'

testing_data_2.loc[(testing_data_2['funder'] == 'Solidame') | (testing_data_2['funder'] == 'Soliderm'), 'funder'] = 'Solidarm'

testing_data_2.loc[(testing_data_2['funder'] == 'Is'), 'funder'] = 'Isf'

testing_data_2.loc[(testing_data_2['funder'] == 'Kilwater'), 'funder'] = 'Killiwater'

testing_data_2.loc[(testing_data_2['funder'] == 'Magadini Makiwaru Water'), 'funder'] = 'Magadini-makiwaru Water'

testing_data_2.loc[(testing_data_2['funder'] == 'Lawate Fuka Water Suppl'), 'funder'] = 'Lawatefuka Water Supply'

testing_data_2.loc[(testing_data_2['funder'] == 'Jika')| (testing_data_2['funder'] == 'Jeica')
                    | (testing_data_2['funder'] == 'Jica'), 'funder'] = 'Jaica'

testing_data_2.loc[(testing_data_2['funder'] == 'Plan Int')| (testing_data_2['funder'] == 'Plan Internatio'), 'funder'] = 'Plan International'

testing_data_2.loc[(testing_data_2['funder'] == 'Dwssp'), 'funder'] = 'Dwsp'

testing_data_2.loc[(testing_data_2['funder'] == 'Fin Water')| (testing_data_2['funder'] == 'Finn Water')
                    | (testing_data_2['funder'] == 'Finw')| (testing_data_2['funder'] == 'Finwater')
                    | (testing_data_2['funder'] == 'Fiwater'), 'funder'] = 'Fini Water'

testing_data_2.loc[(testing_data_2['funder'] == 'Hesaw')| (testing_data_2['funder'] == 'Hesawa')
                    | (testing_data_2['funder'] == 'Hesawwa')| (testing_data_2['funder'] == 'Hesawza')
                    | (testing_data_2['funder'] == 'Hesawza') | (testing_data_2['funder'] == 'Hesswa')
                    | (testing_data_2['funder'] == 'Hewasa')| (testing_data_2['funder'] == 'Hewawa')
                    | (testing_data_2['funder'] == 'Hhesawa'), 'funder'] = 'Hesawa'

testing_data_2.loc[(testing_data_2['funder'] == 'Rural Drinking Water Supply')| (testing_data_2['funder'] == 'Rural Water Department')
                    | (testing_data_2['funder'] == 'Rural Water Supply')| (testing_data_2['funder'] == 'Rural Water Supply And Sanita')
                    | (testing_data_2['funder'] == 'Hesawza') | (testing_data_2['funder'] == 'Rural Water Supply and Sanitat'), 'funder'] = 'Rural Water And Sanitation'

testing_data_2.loc[(testing_data_2['funder'] == 'Quick')| (testing_data_2['funder'] == 'Quick Win')
                    | (testing_data_2['funder'] == 'Quick Win Project')| (testing_data_2['funder'] == 'Quick Win Project/council')
                    | (testing_data_2['funder'] == 'Quick Win/halmashauri') | (testing_data_2['funder'] == 'Quick Wings')
                    | (testing_data_2['funder'] == 'Quick Wins')| (testing_data_2['funder'] == 'Quick Wins Scheme')
                    | (testing_data_2['funder'] == 'Quicklw') | (testing_data_2['funder'] == 'Quickwi')
                    | (testing_data_2['funder'] == 'Quickwins')| (testing_data_2['funder'] == 'Quik')
                    | (testing_data_2['funder'] == 'Qwckwin')| (testing_data_2['funder'] == 'Qwekwin')
                    | (testing_data_2['funder'] == 'Qwick Win') | (testing_data_2['funder'] == 'Qwickwin'), 'funder'] = 'Quwkwin'

testing_data_2.loc[(testing_data_2['funder'] == 'Priva')| (testing_data_2['funder'] == 'Private')
                    | (testing_data_2['funder'] == 'Private Individul')| (testing_data_2['funder'] == 'Private Person'), 'funder'] = 'Private Individual'

testing_data_2.loc[(testing_data_2['funder'] == 'Nerthlands')| (testing_data_2['funder'] == 'Nethalan')
                    | (testing_data_2['funder'] == 'Nethalan')| (testing_data_2['funder'] == 'Netherland')
                    | (testing_data_2['funder'] == 'Netherla'), 'funder'] = 'Netherlands'

testing_data_2.loc[(testing_data_2['funder'] == 'Missi')| (testing_data_2['funder'] == 'Missio')
                    | (testing_data_2['funder'] == 'Missionaries')| (testing_data_2['funder'] == 'Missionary')
               , 'funder'] = 'Mission'

testing_data_2.loc[(testing_data_2['funder'] == 'Unice')| (testing_data_2['funder'] == 'Uniceg')
                    | (testing_data_2['funder'] == 'Unicet')| (testing_data_2['funder'] == 'Unicrf')
                    | (testing_data_2['funder'] == 'Uniseg')
               , 'funder'] = 'Unicef'

testing_data_2.loc[(testing_data_2['funder'] == '0') , 'funder'] = np.nan

#print(testing_data_2['funder'].value_counts())


In [ ]:
#Making installer feature consistent in training data
training_data_2.loc[(training_data_2['installer'] == 'UNICEF') | (training_data_2['installer'] == 'Unisef')
                    | (training_data_2['installer'] == 'UNICRF'), 'installer'] = 'Unicef'


training_data_2.loc[(training_data_2['installer'] == 'World Visiin') | (training_data_2['installer'] == 'World Vission') |(training_data_2['installer'] == 'World vision') |
                    (training_data_2['installer'] == 'Vission')|(training_data_2['installer'] == 'WORLD VISION')
              , 'installer'] = 'World vision'


training_data_2.loc[(training_data_2['installer'] == 'GOVERNMENT'), 'installer'] = 'Government'

 
training_data_2.loc[(training_data_2['installer'] == 'Rc Cathoric') | (training_data_2['installer'] == 'Roman Cathoric') |
                    (training_data_2['installer'] == 'Roman')|(training_data_2['installer'] == 'Roman Cathoic')
                    |(training_data_2['installer'] == 'RC CATHORIC')|(training_data_2['installer'] == 'RC church')
                    |(training_data_2['installer'] == 'Roman cathoic') |(training_data_2['installer'] == 'Roman Cathoic Same')
                    |(training_data_2['installer'] == 'RC C')|(training_data_2['installer'] == 'RC CHURCH')
                    |(training_data_2['installer'] == 'RC Churc')|(training_data_2['installer'] == 'Roman Church')
                    |(training_data_2['installer'] == 'R.C') |(training_data_2['installer'] == 'RC')
                    |(training_data_2['installer'] == 'RC.Church')|(training_data_2['installer'] == 'RC C')
                    |(training_data_2['installer'] == 'RC Ch') |(training_data_2['installer'] == 'RC Church')
                    |(training_data_2['installer'] == 'Roma')|(training_data_2['installer'] == 'Romam')
                    |(training_data_2['installer'] == 'Roman  Catholic') |(training_data_2['installer'] == 'Roman Ca')
                    |(training_data_2['installer'] == 'Roman Cathoic Rulenge Diocese'), 'installer'] = 'Rc Church'


training_data_2.loc[(training_data_2['installer'] == 'District Counci') | (training_data_2['installer'] == 'DISTRICT COUNCIL'), 'installer'] = 'District Council'


training_data_2.loc[(training_data_2['installer'] == 'WORLD BANK') | (training_data_2['installer'] == 'Word Bank') |
                    (training_data_2['installer'] == 'World bank')|(training_data_2['installer'] == 'Wolrd banks'), 'installer'] = 'World Bank'


training_data_2.loc[(training_data_2['installer'] == 'MINSTRYOF WATER'), 'installer'] = 'Ministry of Water'

#Converting - to missing 
training_data_2.loc[(training_data_2['installer'] == '-'), 'installer'] = np.nan

#Making Kkkt consistent in the insaller column 
training_data_2.loc[(training_data_2['installer'] == 'KKKT _ Konde and DWE') | (training_data_2['installer'] == 'KKKT CHURCH') |
                    (training_data_2['installer'] == 'KKKT-dioces ya Pare')|(training_data_2['installer'] == 'KkKT')|(training_data_2['installer'] == 'Kkkt'), 'installer'] = 'KKKT'

training_data_2.loc[(training_data_2['installer'] == 'Arisan') | (training_data_2['installer'] == 'Arrian') |
                    (training_data_2['installer'] == 'Atisan'), 'installer'] = 'Artisan'

training_data_2.loc[(training_data_2['installer'] == 'Cebtral Government') | (training_data_2['installer'] == 'Cental Government') 
                    |(training_data_2['installer'] == 'Centr')|(training_data_2['installer'] == 'Centra govt')
                    |(training_data_2['installer'] == 'Centra govt')|(training_data_2['installer'] == 'Central govt'), 'installer'] = 'Central government'


training_data_2.loc[(training_data_2['installer'] == 'Commu') | (training_data_2['installer'] == 'Communit') 
                    |(training_data_2['installer'] == 'Comunity'), 'installer'] = 'Community'

training_data_2.loc[(training_data_2['installer'] == 'Consultant Engineer') | (training_data_2['installer'] == 'Consulting Engineer') 
                    , 'installer'] = 'Consulting Engineer'

training_data_2.loc[(training_data_2['installer'] == 'DANIDA') | (training_data_2['installer'] == 'Danid') 
                    | (training_data_2['installer'] == 'DANIDA CO'), 'installer'] = 'Danida'

training_data_2.loc[(training_data_2['installer'] == 'Distric Water Department') | (training_data_2['installer'] == 'District water depar') 
                    , 'installer'] = 'District water department'

training_data_2.loc[(training_data_2['installer'] == 'Distric Council') | (training_data_2['installer'] == 'District Counci') 
                    , 'installer'] = 'District council'

training_data_2.loc[(training_data_2['installer'] == 'DW') | (training_data_2['installer'] == 'DW E') | (training_data_2['installer'] == 'DW#')
                    | (training_data_2['installer'] == 'DW$') | (training_data_2['installer'] == 'DWE&') | (training_data_2['installer'] == 'DWE/')
                    | (training_data_2['installer'] == 'DWE}')| (training_data_2['installer'] == 'DWR')  
                    , 'installer'] = 'DWE'

training_data_2.loc[(training_data_2['installer'] == 'FinWater')| (training_data_2['installer'] == 'FINN WATER')
                    | (training_data_2['installer'] == 'FinW')| (training_data_2['installer'] == 'FinWater')
                    | (training_data_2['installer'] == 'FinWate') | (training_data_2['installer'] == 'FIN WATER')
                    | (training_data_2['installer'] == 'FW'), 'installer'] = 'Fini water'

training_data_2.loc[(training_data_2['installer'] == 'Got')| (training_data_2['installer'] == 'Gove')
                    | (training_data_2['installer'] == 'Gover')| (training_data_2['installer'] == 'GOVERM')
                    | (training_data_2['installer'] == 'GOVERN') | (training_data_2['installer'] == 'GOVERNME')
                    | (training_data_2['installer'] == 'Governmen'), 'installer'] = 'Government'

training_data_2.loc[(training_data_2['installer'] == 'Handeni Truck Main('), 'installer'] = 'Handeni Truck Main'

training_data_2.loc[(training_data_2['installer'] == 'HEESAW') | (training_data_2['installer'] == 'HESAW')
                    | (training_data_2['installer'] == 'HESAWQ')| (training_data_2['installer'] == 'HESAWS') 
                    | (training_data_2['installer'] == 'HESAWZ') 
                    , 'installer'] = 'HESAWA'

training_data_2.loc[(training_data_2['installer'] == 'JAICA CO') | (training_data_2['installer'] == 'JALCA')
                    | (training_data_2['installer'] == 'Jeica')| (training_data_2['installer'] == 'JICA')
                    | (training_data_2['installer'] == 'JIKA')
                  , 'installer'] = 'JAICA'

training_data_2.loc[(training_data_2['installer'] == 'KILI WATER') | (training_data_2['installer'] == 'Kiliwater r')
                    | (training_data_2['installer'] == 'Kilwater') 
                    , 'installer'] = 'Kiliwater'

training_data_2.loc[(training_data_2['installer'] == 'KK') | (training_data_2['installer'] == 'KKT')
                    | (training_data_2['installer'] == 'KKT C') 
                    , 'installer'] = 'KKKT'

training_data_2.loc[(training_data_2['installer'] == 'Kuwaiti') | (training_data_2['installer'] == 'Kuweit'), 'installer'] = 'KUWAIT'

training_data_2.loc[(training_data_2['installer'] == 'Lawate fuka water su') | (training_data_2['installer'] == 'Lawatefuka water sup'), 'installer'] = 'Lawatefuka water supply'

training_data_2.loc[(training_data_2['installer'] == 'Local technician') | (training_data_2['installer'] == 'Local l technician')
                    | (training_data_2['installer'] == 'Local te')  | (training_data_2['installer'] == 'Local technical') 
                    | (training_data_2['installer'] == 'Local technical tec') | (training_data_2['installer'] == 'Local technician')
                    | (training_data_2['installer'] == 'Local technitian') | (training_data_2['installer'] == 'Locall technician')
                    | (training_data_2['installer'] == 'Localtechnician') 
                    , 'installer'] = 'Local Technician'

training_data_2.loc[(training_data_2['installer'] == 'Magadini Makiwaru wa'), 'installer'] = 'Magadini-Makiwaru wa'

training_data_2.loc[(training_data_2['installer'] == 'Missi')| (training_data_2['installer'] == 'Missio')
                    | (training_data_2['installer'] == 'Missionaries')| (training_data_2['installer'] == 'Missionary')
               , 'installer'] = 'Mission'

training_data_2.loc[(training_data_2['installer'] == 'MWE &'), 'installer'] = 'MWE'

training_data_2.loc[(training_data_2['installer'] == 'NORA')| (training_data_2['installer'] == 'NORAD/')
                    , 'installer'] = 'Norad'

training_data_2.loc[(training_data_2['installer'] == 'Not kno')| (training_data_2['installer'] == 'Not known')
                    , 'installer'] = " "

training_data_2.loc[(training_data_2['installer'] == 'OIKOS')| (training_data_2['installer'] == 'Oikos E Africa')
                    | (training_data_2['installer'] == 'Oikos E.Africa')| (training_data_2['installer'] == 'Oikos E.Afrika')
               , 'installer'] = 'Oikos E. Africa'

training_data_2.loc[(training_data_2['installer'] == 'OXFARM')
                    , 'installer'] = "OXFAM"

training_data_2.loc[(training_data_2['installer'] == 'PRIV')| (training_data_2['installer'] == 'Priva')| (training_data_2['installer'] == 'Privat')
, 'installer'] = "Private"

training_data_2.loc[(training_data_2['installer'] == 'Quick')| (training_data_2['funder'] == 'Quick Win')
                    | (training_data_2['installer'] == 'Quick Win Project')| (training_data_2['funder'] == 'Quick Win Project/council')
                    | (training_data_2['installer'] == 'Quick Win/halmashauri') | (training_data_2['funder'] == 'Quick Wings')
                    | (training_data_2['installer'] == 'Quick Wins')| (training_data_2['funder'] == 'Quick Wins Scheme')
                    | (training_data_2['installer'] == 'Quicklw') | (training_data_2['funder'] == 'Quickwi')
                    | (training_data_2['installer'] == 'Quickwins')| (training_data_2['funder'] == 'Quik')
                    | (training_data_2['installer'] == 'Qwckwin')| (training_data_2['funder'] == 'Qwekwin')
                    | (training_data_2['installer'] == 'Qwick Win') | (training_data_2['funder'] == 'Qwickwin'), 'funder'] = 'Quwkwin'

training_data_2.loc[(training_data_2['installer'] == 'Region Water') | (training_data_2['installer'] == 'Regional Water')
                    , 'installer'] = 'Regional Water Department'


training_data_2.loc[(training_data_2['installer'] == 'RWSP'), 'installer'] = "RWSSP"

training_data_2.loc[(training_data_2['installer'] == 'SEMA CO LTD')| (training_data_2['installer'] == 'SEMA Consultant')
                    , 'installer'] = "SEMA"

training_data_2.loc[(training_data_2['installer'] == 'SHIP')| (training_data_2['installer'] == 'SHIPO CONSTRUCTORS')
                    , 'installer'] = "SHIPO"

training_data_2.loc[(training_data_2['installer'] == 'TASA')| (training_data_2['installer'] == 'TASAF 1')|(training_data_2['installer'] == 'TASAF/')
                    |(training_data_2['installer'] == 'TASF')|(training_data_2['installer'] == 'TASSAF'), 'installer'] = "TASAF"

training_data_2.loc[(training_data_2['installer'] == 'TCRS a'), 'installer'] = "TCRS"

training_data_2.loc[(training_data_2['installer'] == 'TWESS'), 'installer'] = "TWESA"

training_data_2.loc[(training_data_2['installer'] == 'wanan'), 'installer'] = "wananchi"

training_data_2.loc[(training_data_2['installer'] == 'WATER AID')| (training_data_2['installer'] == 'WATER AIDS')|(training_data_2['installer'] == 'WATERAID')
                    , 'installer'] = "Water Aid"

training_data_2.loc[(training_data_2['installer'] == 'WDECO')| (training_data_2['installer'] == 'WEDEKO'), 'installer'] = "WEDECO"

training_data_2.loc[(training_data_2['installer'] == 'Wizara ya maji')| (training_data_2['installer'] == 'WIZARA'), 'installer'] = "Wizara ya maji"

training_data_2.loc[(training_data_2['installer'] == 'Word Bank')| (training_data_2['installer'] == 'World banks')|(training_data_2['installer'] == 'WOULD BANK')
                    , 'installer'] = "World Bank"

#print(training_data_2['installer'].value_counts())


In [ ]:
#Making installer feature consistent in testing data
testing_data_2.loc[(testing_data_2['installer'] == 'UNICEF') | (testing_data_2['installer'] == 'Unisef')
                    | (testing_data_2['installer'] == 'UNICRF'), 'installer'] = 'Unicef'


testing_data_2.loc[(testing_data_2['installer'] == 'World Visiin') | (testing_data_2['installer'] == 'World Vission') |(testing_data_2['installer'] == 'World vision') |
                    (testing_data_2['installer'] == 'Vission')|(testing_data_2['installer'] == 'WORLD VISION')
              , 'installer'] = 'World vision'


testing_data_2.loc[(testing_data_2['installer'] == 'GOVERNMENT'), 'installer'] = 'Government'

 
testing_data_2.loc[(testing_data_2['installer'] == 'Rc Cathoric') | (testing_data_2['installer'] == 'Roman Cathoric') |
                    (testing_data_2['installer'] == 'Roman')|(testing_data_2['installer'] == 'Roman Cathoic')
                    |(testing_data_2['installer'] == 'RC CATHORIC')|(testing_data_2['installer'] == 'RC church')
                    |(testing_data_2['installer'] == 'Roman cathoic') |(testing_data_2['installer'] == 'Roman Cathoic Same')
                    |(testing_data_2['installer'] == 'RC C')|(testing_data_2['installer'] == 'RC CHURCH')
                    |(testing_data_2['installer'] == 'RC Churc')|(testing_data_2['installer'] == 'Roman Church')
                    |(testing_data_2['installer'] == 'R.C') |(testing_data_2['installer'] == 'RC')
                    |(testing_data_2['installer'] == 'RC.Church')|(testing_data_2['installer'] == 'RC C')
                    |(testing_data_2['installer'] == 'RC Ch') |(testing_data_2['installer'] == 'RC Church')
                    |(testing_data_2['installer'] == 'Roma')|(testing_data_2['installer'] == 'Romam')
                    |(testing_data_2['installer'] == 'Roman  Catholic') |(testing_data_2['installer'] == 'Roman Ca')
                    |(testing_data_2['installer'] == 'Roman Cathoic Rulenge Diocese'), 'installer'] = 'Rc Church'


testing_data_2.loc[(testing_data_2['installer'] == 'District Counci') | (testing_data_2['installer'] == 'DISTRICT COUNCIL'), 'installer'] = 'District Council'


testing_data_2.loc[(testing_data_2['installer'] == 'WORLD BANK') | (testing_data_2['installer'] == 'Word Bank') |
                    (testing_data_2['installer'] == 'World bank')|(testing_data_2['installer'] == 'Wolrd banks'), 'installer'] = 'World Bank'


testing_data_2.loc[(testing_data_2['installer'] == 'MINSTRYOF WATER'), 'installer'] = 'Ministry of Water'

#Converting - to missing 
testing_data_2.loc[(testing_data_2['installer'] == '-'), 'installer'] = np.nan

#Making Kkkt consistent in the insaller column 
testing_data_2.loc[(testing_data_2['installer'] == 'KKKT _ Konde and DWE') | (testing_data_2['installer'] == 'KKKT CHURCH') |
                    (testing_data_2['installer'] == 'KKKT-dioces ya Pare')|(testing_data_2['installer'] == 'KkKT')|(testing_data_2['installer'] == 'Kkkt'), 'installer'] = 'KKKT'

testing_data_2.loc[(testing_data_2['installer'] == 'Arisan') | (testing_data_2['installer'] == 'Arrian') |
                    (testing_data_2['installer'] == 'Atisan'), 'installer'] = 'Artisan'

testing_data_2.loc[(testing_data_2['installer'] == 'Cebtral Government') | (testing_data_2['installer'] == 'Cental Government') 
                    |(testing_data_2['installer'] == 'Centr')|(testing_data_2['installer'] == 'Centra govt')
                    |(testing_data_2['installer'] == 'Centra govt')|(testing_data_2['installer'] == 'Central govt'), 'installer'] = 'Central government'


testing_data_2.loc[(testing_data_2['installer'] == 'Commu') | (testing_data_2['installer'] == 'Communit') 
                    |(testing_data_2['installer'] == 'Comunity'), 'installer'] = 'Community'

testing_data_2.loc[(testing_data_2['installer'] == 'Consultant Engineer') | (testing_data_2['installer'] == 'Consulting Engineer') 
                    , 'installer'] = 'Consulting Engineer'

testing_data_2.loc[(testing_data_2['installer'] == 'DANIDA') | (testing_data_2['installer'] == 'Danid') 
                    | (testing_data_2['installer'] == 'DANIDA CO'), 'installer'] = 'Danida'

testing_data_2.loc[(testing_data_2['installer'] == 'Distric Water Department') | (testing_data_2['installer'] == 'District water depar') 
                    , 'installer'] = 'District water department'

testing_data_2.loc[(testing_data_2['installer'] == 'Distric Council') | (testing_data_2['installer'] == 'District Counci') 
                    , 'installer'] = 'District council'

testing_data_2.loc[(testing_data_2['installer'] == 'DW') | (testing_data_2['installer'] == 'DW E') | (testing_data_2['installer'] == 'DW#')
                    | (testing_data_2['installer'] == 'DW$') | (testing_data_2['installer'] == 'DWE&') | (testing_data_2['installer'] == 'DWE/')
                    | (testing_data_2['installer'] == 'DWE}')| (testing_data_2['installer'] == 'DWR')  
                    , 'installer'] = 'DWE'

testing_data_2.loc[(testing_data_2['installer'] == 'FinWater')| (testing_data_2['installer'] == 'FINN WATER')
                    | (testing_data_2['installer'] == 'FinW')| (testing_data_2['installer'] == 'FinWater')
                    | (testing_data_2['installer'] == 'FinWate') | (testing_data_2['installer'] == 'FIN WATER')
                    | (testing_data_2['installer'] == 'FW'), 'installer'] = 'Fini water'

testing_data_2.loc[(testing_data_2['installer'] == 'Got')| (testing_data_2['installer'] == 'Gove')
                    | (testing_data_2['installer'] == 'Gover')| (testing_data_2['installer'] == 'GOVERM')
                    | (testing_data_2['installer'] == 'GOVERN') | (testing_data_2['installer'] == 'GOVERNME')
                    | (testing_data_2['installer'] == 'Governmen'), 'installer'] = 'Government'

testing_data_2.loc[(testing_data_2['installer'] == 'Handeni Truck Main('), 'installer'] = 'Handeni Truck Main'

testing_data_2.loc[(testing_data_2['installer'] == 'HEESAW') | (testing_data_2['installer'] == 'HESAW')
                    | (testing_data_2['installer'] == 'HESAWQ')| (testing_data_2['installer'] == 'HESAWS') 
                    | (testing_data_2['installer'] == 'HESAWZ') 
                    , 'installer'] = 'HESAWA'

testing_data_2.loc[(testing_data_2['installer'] == 'JAICA CO') | (testing_data_2['installer'] == 'JALCA')
                    | (testing_data_2['installer'] == 'Jeica')| (testing_data_2['installer'] == 'JICA')
                    | (testing_data_2['installer'] == 'JIKA')
                  , 'installer'] = 'JAICA'

testing_data_2.loc[(testing_data_2['installer'] == 'KILI WATER') | (testing_data_2['installer'] == 'Kiliwater r')
                    | (testing_data_2['installer'] == 'Kilwater') 
                    , 'installer'] = 'Kiliwater'

testing_data_2.loc[(testing_data_2['installer'] == 'KK') | (testing_data_2['installer'] == 'KKT')
                    | (testing_data_2['installer'] == 'KKT C') 
                    , 'installer'] = 'KKKT'

testing_data_2.loc[(testing_data_2['installer'] == 'Kuwaiti') | (testing_data_2['installer'] == 'Kuweit'), 'installer'] = 'KUWAIT'

testing_data_2.loc[(testing_data_2['installer'] == 'Lawate fuka water su') | (testing_data_2['installer'] == 'Lawatefuka water sup'), 'installer'] = 'Lawatefuka water supply'

testing_data_2.loc[(testing_data_2['installer'] == 'Local technician') | (testing_data_2['installer'] == 'Local l technician')
                    | (testing_data_2['installer'] == 'Local te')  | (testing_data_2['installer'] == 'Local technical') 
                    | (testing_data_2['installer'] == 'Local technical tec') | (testing_data_2['installer'] == 'Local technician')
                    | (testing_data_2['installer'] == 'Local technitian') | (testing_data_2['installer'] == 'Locall technician')
                    | (testing_data_2['installer'] == 'Localtechnician') 
                    , 'installer'] = 'Local Technician'

testing_data_2.loc[(testing_data_2['installer'] == 'Magadini Makiwaru wa'), 'installer'] = 'Magadini-Makiwaru wa'

testing_data_2.loc[(testing_data_2['installer'] == 'Missi')| (testing_data_2['installer'] == 'Missio')
                    | (testing_data_2['installer'] == 'Missionaries')| (testing_data_2['installer'] == 'Missionary')
               , 'installer'] = 'Mission'

testing_data_2.loc[(testing_data_2['installer'] == 'MWE &'), 'installer'] = 'MWE'

testing_data_2.loc[(testing_data_2['installer'] == 'NORA')| (testing_data_2['installer'] == 'NORAD/')
                    , 'installer'] = 'Norad'

testing_data_2.loc[(testing_data_2['installer'] == 'Not kno')| (testing_data_2['installer'] == 'Not known')
                    , 'installer'] = " "

testing_data_2.loc[(testing_data_2['installer'] == 'OIKOS')| (testing_data_2['installer'] == 'Oikos E Africa')
                    | (testing_data_2['installer'] == 'Oikos E.Africa')| (testing_data_2['installer'] == 'Oikos E.Afrika')
               , 'installer'] = 'Oikos E. Africa'

testing_data_2.loc[(testing_data_2['installer'] == 'OXFARM')
                    , 'installer'] = "OXFAM"

testing_data_2.loc[(testing_data_2['installer'] == 'PRIV')| (testing_data_2['installer'] == 'Priva')| (testing_data_2['installer'] == 'Privat')
, 'installer'] = "Private"

testing_data_2.loc[(testing_data_2['installer'] == 'Quick')| (testing_data_2['funder'] == 'Quick Win')
                    | (testing_data_2['installer'] == 'Quick Win Project')| (testing_data_2['funder'] == 'Quick Win Project/council')
                    | (testing_data_2['installer'] == 'Quick Win/halmashauri') | (testing_data_2['funder'] == 'Quick Wings')
                    | (testing_data_2['installer'] == 'Quick Wins')| (testing_data_2['funder'] == 'Quick Wins Scheme')
                    | (testing_data_2['installer'] == 'Quicklw') | (testing_data_2['funder'] == 'Quickwi')
                    | (testing_data_2['installer'] == 'Quickwins')| (testing_data_2['funder'] == 'Quik')
                    | (testing_data_2['installer'] == 'Qwckwin')| (testing_data_2['funder'] == 'Qwekwin')
                    | (testing_data_2['installer'] == 'Qwick Win') | (testing_data_2['funder'] == 'Qwickwin'), 'funder'] = 'Quwkwin'

testing_data_2.loc[(testing_data_2['installer'] == 'Region Water') | (testing_data_2['installer'] == 'Regional Water')
                    , 'installer'] = 'Regional Water Department'


testing_data_2.loc[(testing_data_2['installer'] == 'RWSP'), 'installer'] = "RWSSP"

testing_data_2.loc[(testing_data_2['installer'] == 'SEMA CO LTD')| (testing_data_2['installer'] == 'SEMA Consultant')
                    , 'installer'] = "SEMA"

testing_data_2.loc[(testing_data_2['installer'] == 'SHIP')| (testing_data_2['installer'] == 'SHIPO CONSTRUCTORS')
                    , 'installer'] = "SHIPO"

testing_data_2.loc[(testing_data_2['installer'] == 'TASA')| (testing_data_2['installer'] == 'TASAF 1')|(testing_data_2['installer'] == 'TASAF/')
                    |(testing_data_2['installer'] == 'TASF')|(testing_data_2['installer'] == 'TASSAF'), 'installer'] = "TASAF"

testing_data_2.loc[(testing_data_2['installer'] == 'TCRS a'), 'installer'] = "TCRS"

testing_data_2.loc[(testing_data_2['installer'] == 'TWESS'), 'installer'] = "TWESA"

testing_data_2.loc[(testing_data_2['installer'] == 'wanan'), 'installer'] = "wananchi"

testing_data_2.loc[(testing_data_2['installer'] == 'WATER AID')| (testing_data_2['installer'] == 'WATER AIDS')|(testing_data_2['installer'] == 'WATERAID')
                    , 'installer'] = "Water Aid"

testing_data_2.loc[(testing_data_2['installer'] == 'WDECO')| (testing_data_2['installer'] == 'WEDEKO'), 'installer'] = "WEDECO"

testing_data_2.loc[(testing_data_2['installer'] == 'Wizara ya maji')| (testing_data_2['installer'] == 'WIZARA'), 'installer'] = "Wizara ya maji"

testing_data_2.loc[(testing_data_2['installer'] == 'Word Bank')| (testing_data_2['installer'] == 'World banks')|(testing_data_2['installer'] == 'WOULD BANK')
                    , 'installer'] = "World Bank"

#print(testing_data_2['installer'].value_counts())

In [ ]:
#Making scheme name consistent for training data
training_data_2.loc[(training_data_2['scheme_name'] == "wanging'ombe water supply s"), 'scheme_name'] = "wanging'ombe supply scheme"

training_data_2.loc[(training_data_2['scheme_name'] == "Chalinze wate"), 'scheme_name'] = "Chalinze water"

training_data_2.loc[(training_data_2['scheme_name'] == "Bagamoyo wate"), 'scheme_name'] = "Bagamoyo water"

training_data_2.loc[(training_data_2['scheme_name'] == "Darida"), 'scheme_name'] = "DANIDA"

training_data_2.loc[(training_data_2['scheme_name'] == "Uroki-Bomang'ombe water sup"), 'scheme_name'] = "Uroki-Bomang'ombe water suppply"

training_data_2.loc[(training_data_2['scheme_name'] == "Sinyanga  water supplied sc"), 'scheme_name'] = "Sinyanga  water supplied scheme"

training_data_2.loc[(training_data_2['scheme_name'] == "Ngamanga water supplied sch"), 'scheme_name'] = "Ngamanga water supplied scheme"

training_data_2.loc[(training_data_2['scheme_name'] == "imalinyi water supply schem"), 'scheme_name'] = "imalinyi water supply scheme"

training_data_2.loc[(training_data_2['scheme_name'] == "matembwe water supply schem"), 'scheme_name'] = "matembwe water supply scheme"

training_data_2.loc[(training_data_2['scheme_name'] == "ngamanga water supplied sch"), 'scheme_name'] = "ngamanga water supplied scheme"

training_data_2.loc[(training_data_2['scheme_name'] == "Igongolo gravity water sche"), 'scheme_name'] = "Igongolo gravity water scheme"

training_data_2.loc[(training_data_2['scheme_name'] == "Mtwango water supplied sche"), 'scheme_name'] = "Mtwango water supplied scheme"
  
training_data_2.loc[(training_data_2['scheme_name'] == "Kishoju Water  Su "), 'scheme_name'] = "Kishoju Water supply"

training_data_2.loc[(training_data_2['scheme_name'] == "Nkwenda water sup  "), 'scheme_name'] = "Nkwenda water supply"

training_data_2.loc[(training_data_2['scheme_name'] == "Kwamazandu water sup"), 'scheme_name'] = "Kwamazandu water supply"

training_data_2.loc[(training_data_2['scheme_name'] == "Bore Hole")| (training_data_2['scheme_name'] == 'Borehole drilling project')
                    | (training_data_2['scheme_name'] == 'Borehole pump pumped scheme')| (training_data_2['scheme_name'] == 'Borehole pumped scheme'), 'scheme_name'] = "Borehole"

training_data_2.loc[(training_data_2['scheme_name'] == "Handeni Trunk Mai (H")| (training_data_2['scheme_name'] == 'Handeni Truck Main (')
                    | (training_data_2['scheme_name'] == 'Handeni Trunk Main(H')| (training_data_2['scheme_name'] == 'Handeni Trunk Maini('), 'scheme_name'] = "Handeni Trunk Maini"

training_data_2.loc[(training_data_2['scheme_name'] == "imalinyi supply scheme")|(training_data_2['scheme_name'] == "imalinyi water supply schem"), 'scheme_name'] = "imalinyi water supply scheme"

training_data_2.loc[(training_data_2['scheme_name'] == "Jumuhiya ya watumia  maji")| (training_data_2['scheme_name'] == 'Jumuiya ya watumia maji kilema kusini')
                    | (training_data_2['scheme_name'] == 'Jumwia ya wa tumia maji Vumamti'), 'scheme_name'] = "Jumuhiya ya watumia maji"

training_data_2.loc[(training_data_2['scheme_name'] == "Losaa kia gravity water supply")|(training_data_2['scheme_name'] == "Losaa gravity water supply")
                    |(training_data_2['scheme_name'] == "Losaa Kia water supply"), 'scheme_name'] = "Losaa-Kia water supply"

training_data_2.loc[(training_data_2['scheme_name'] == "Lyamungo umbwe water supply"), 'scheme_name'] = "Lyamungo-Umbwe water supply"

training_data_2.loc[(training_data_2['scheme_name'] == "Maambreni gravity water supply breni"), 'scheme_name'] = "Maambreni gravity water supply"

training_data_2.loc[(training_data_2['scheme_name'] == "Mkongoro one water project"), 'scheme_name'] = "Mkongoro One"

training_data_2.loc[(training_data_2['scheme_name'] == "Mkongoro  Two"), 'scheme_name'] = "Mkongoro Two"

training_data_2.loc[(training_data_2['scheme_name'] == "Mtwango water supplied sche"), 'scheme_name'] = "mtwango water supply scheme"

training_data_2.loc[(training_data_2['scheme_name'] == "ngamanga  water supplied sc")|(training_data_2['scheme_name'] == "Ngamanga water supplied sch")
                |(training_data_2['scheme_name'] == "Ngamanga water supply schem"), 'scheme_name'] = "Ngamanga water supply scheme"

training_data_2.loc[(training_data_2['scheme_name'] == "None")|(training_data_2['scheme_name'] == "not known"), 'scheme_name'] = np.nan

training_data_2.loc[(training_data_2['scheme_name'] == "Olkokola mwandet pipe line")|(training_data_2['scheme_name'] == "Olkokola pipe line")
                |(training_data_2['scheme_name'] == "Olkokola-mwandet pipe line"), 'scheme_name'] = "Olkokola-mwandet pipeline"

training_data_2.loc[(training_data_2['scheme_name'] == "RC")|(training_data_2['scheme_name'] == "RC CHU")
                |(training_data_2['scheme_name'] == "RC Missionary")
                |(training_data_2['scheme_name'] == "RC Msufi")
                |(training_data_2['scheme_name'] == "RC WATER SUPPLY")
                |(training_data_2['scheme_name'] == "Roman"), 'scheme_name'] = "Rc Church"

training_data_2.loc[(training_data_2['scheme_name'] == "Shallow Welll"), 'scheme_name'] = "Shallow well"

training_data_2.loc[(training_data_2['scheme_name'] == "Sinyanga  water supplied sc")
                    |(training_data_2['scheme_name'] == "Sinyanga water supplied sch"), 'scheme_name'] = "Sinyanga water supply scheme"

training_data_2.loc[(training_data_2['scheme_name'] == "Tove")
                    |(training_data_2['scheme_name'] == "Tove - Mtwango")
                    |(training_data_2['scheme_name'] == "Tove mtwango")
                    |(training_data_2['scheme_name'] == "Tove -mtwango")
                    |(training_data_2['scheme_name'] == "Tove- mtwango")
                    |(training_data_2['scheme_name'] == "Tove Mtwango gravity Scheme")
                    |(training_data_2['scheme_name'] == "tove mtwango water supply s")
                    |(training_data_2['scheme_name'] == "Tove-mtwango"), 'scheme_name'] = "Tove-mtwango water supply scheme"

training_data_2.loc[(training_data_2['scheme_name'] == "Upper  ruvu")
                    |(training_data_2['scheme_name'] == "Upper River")
                    |(training_data_2['scheme_name'] == "upper Ruv")
                    |(training_data_2['scheme_name'] == "Upper Ruvu Ba")
                    |(training_data_2['scheme_name'] == "upper Ruvuv"), 'scheme_name'] = "Upper Ruvu"

training_data_2.loc[(training_data_2['scheme_name'] == "Uroki Bomang'ombe water sup")
                    |(training_data_2['scheme_name'] == "Uroki-Bomang'ombe water sup"), 'scheme_name'] = "Uroki-Bomang'ombe water supply"

training_data_2.loc[(training_data_2['scheme_name'] == "wanging'ombe supply scheme")
                    |(training_data_2['scheme_name'] == "wanging'ombe water supply s")
                    |(training_data_2['scheme_name'] == "wanginyi water supplied sch"),'scheme_name'] = "wanging'ombe water supply scheme"
    
training_data_2.loc[(training_data_2['scheme_name'] == "World Bank Water Project")
                    |(training_data_2['scheme_name'] == "World Bank Water Supply")
                    |(training_data_2['scheme_name'] == "World banks")
                    |(training_data_2['scheme_name'] == "World banks Water board")
                    |(training_data_2['scheme_name'] == "World banks water supplying")
                    |(training_data_2['scheme_name'] == "WOULD BANK"), 'scheme_name'] = "World Bank"                     
                      
training_data_2.loc[(training_data_2['scheme_name'] == "K")
                    |(training_data_2['scheme_name'] == "M")
                    |(training_data_2['scheme_name'] == "N")
                    |(training_data_2['scheme_name'] == "S") 
                    |(training_data_2['scheme_name'] == "D")
                    |(training_data_2['scheme_name'] == "G")
                    |(training_data_2['scheme_name'] == "J") 
                    |(training_data_2['scheme_name'] == "U")
                    |(training_data_2['scheme_name'] == "A")
                    |(training_data_2['scheme_name'] == "W")
                    |(training_data_2['scheme_name'] == "no scheme") 
                    |(training_data_2['scheme_name'] == "B") 
                    |(training_data_2['scheme_name'] == "I"), 'scheme_name']= np.nan

training_data_2.loc[(training_data_2['scheme_name'] == "Makwale water supplied sche"), 'scheme_name'] = "makwale water supplied scheme"
                     
#print(training_data_2['scheme_name'].value_counts())


In [ ]:
#Making scheme name consistent for testing data
testing_data_2.loc[(testing_data_2['scheme_name'] == "wanging'ombe water supply s"), 'scheme_name'] = "wanging'ombe supply scheme"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Chalinze wate"), 'scheme_name'] = "Chalinze water"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Bagamoyo wate"), 'scheme_name'] = "Bagamoyo water"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Darida"), 'scheme_name'] = "DANIDA"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Uroki-Bomang'ombe water sup"), 'scheme_name'] = "Uroki-Bomang'ombe water suppply"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Sinyanga  water supplied sc"), 'scheme_name'] = "Sinyanga  water supplied scheme"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Ngamanga water supplied sch"), 'scheme_name'] = "Ngamanga water supplied scheme"

testing_data_2.loc[(testing_data_2['scheme_name'] == "imalinyi water supply schem"), 'scheme_name'] = "imalinyi water supply scheme"

testing_data_2.loc[(testing_data_2['scheme_name'] == "matembwe water supply schem"), 'scheme_name'] = "matembwe water supply scheme"

testing_data_2.loc[(testing_data_2['scheme_name'] == "ngamanga water supplied sch"), 'scheme_name'] = "ngamanga water supplied scheme"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Igongolo gravity water sche"), 'scheme_name'] = "Igongolo gravity water scheme"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Mtwango water supplied sche"), 'scheme_name'] = "Mtwango water supplied scheme"
  
testing_data_2.loc[(testing_data_2['scheme_name'] == "Kishoju Water  Su "), 'scheme_name'] = "Kishoju Water supply"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Nkwenda water sup  "), 'scheme_name'] = "Nkwenda water supply"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Kwamazandu water sup"), 'scheme_name'] = "Kwamazandu water supply"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Bore Hole")| (testing_data_2['scheme_name'] == 'Borehole drilling project')
                    | (testing_data_2['scheme_name'] == 'Borehole pump pumped scheme')| (testing_data_2['scheme_name'] == 'Borehole pumped scheme'), 'scheme_name'] = "Borehole"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Handeni Trunk Mai (H")| (testing_data_2['scheme_name'] == 'Handeni Truck Main (')
                    | (testing_data_2['scheme_name'] == 'Handeni Trunk Main(H')| (testing_data_2['scheme_name'] == 'Handeni Trunk Maini('), 'scheme_name'] = "Handeni Trunk Maini"

testing_data_2.loc[(testing_data_2['scheme_name'] == "imalinyi supply scheme")|(testing_data_2['scheme_name'] == "imalinyi water supply schem"), 'scheme_name'] = "imalinyi water supply scheme"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Jumuhiya ya watumia  maji")| (testing_data_2['scheme_name'] == 'Jumuiya ya watumia maji kilema kusini')
                    | (testing_data_2['scheme_name'] == 'Jumwia ya wa tumia maji Vumamti'), 'scheme_name'] = "Jumuhiya ya watumia maji"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Losaa kia gravity water supply")|(testing_data_2['scheme_name'] == "Losaa gravity water supply")
                    |(testing_data_2['scheme_name'] == "Losaa Kia water supply"), 'scheme_name'] = "Losaa-Kia water supply"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Lyamungo umbwe water supply"), 'scheme_name'] = "Lyamungo-Umbwe water supply"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Maambreni gravity water supply breni"), 'scheme_name'] = "Maambreni gravity water supply"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Mkongoro one water project"), 'scheme_name'] = "Mkongoro One"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Mkongoro  Two"), 'scheme_name'] = "Mkongoro Two"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Mtwango water supplied sche"), 'scheme_name'] = "mtwango water supply scheme"

testing_data_2.loc[(testing_data_2['scheme_name'] == "ngamanga  water supplied sc")|(testing_data_2['scheme_name'] == "Ngamanga water supplied sch")
                |(testing_data_2['scheme_name'] == "Ngamanga water supply schem"), 'scheme_name'] = "Ngamanga water supply scheme"

testing_data_2.loc[(testing_data_2['scheme_name'] == "None")|(testing_data_2['scheme_name'] == "not known"), 'scheme_name'] = np.nan

testing_data_2.loc[(testing_data_2['scheme_name'] == "Olkokola mwandet pipe line")|(testing_data_2['scheme_name'] == "Olkokola pipe line")
                |(testing_data_2['scheme_name'] == "Olkokola-mwandet pipe line"), 'scheme_name'] = "Olkokola-mwandet pipeline"

testing_data_2.loc[(testing_data_2['scheme_name'] == "RC")|(testing_data_2['scheme_name'] == "RC CHU")
                |(testing_data_2['scheme_name'] == "RC Missionary")
                |(testing_data_2['scheme_name'] == "RC Msufi")
                |(testing_data_2['scheme_name'] == "RC WATER SUPPLY")
                |(testing_data_2['scheme_name'] == "Roman"), 'scheme_name'] = "Rc Church"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Shallow Welll"), 'scheme_name'] = "Shallow well"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Sinyanga  water supplied sc")
                    |(testing_data_2['scheme_name'] == "Sinyanga water supplied sch"), 'scheme_name'] = "Sinyanga water supply scheme"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Tove")
                    |(testing_data_2['scheme_name'] == "Tove - Mtwango")
                    |(testing_data_2['scheme_name'] == "Tove mtwango")
                    |(testing_data_2['scheme_name'] == "Tove -mtwango")
                    |(testing_data_2['scheme_name'] == "Tove- mtwango")
                    |(testing_data_2['scheme_name'] == "Tove Mtwango gravity Scheme")
                    |(testing_data_2['scheme_name'] == "tove mtwango water supply s")
                    |(testing_data_2['scheme_name'] == "Tove-mtwango"), 'scheme_name'] = "Tove-mtwango water supply scheme"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Upper  ruvu")
                    |(testing_data_2['scheme_name'] == "Upper River")
                    |(testing_data_2['scheme_name'] == "upper Ruv")
                    |(testing_data_2['scheme_name'] == "Upper Ruvu Ba")
                    |(testing_data_2['scheme_name'] == "upper Ruvuv"), 'scheme_name'] = "Upper Ruvu"

testing_data_2.loc[(testing_data_2['scheme_name'] == "Uroki Bomang'ombe water sup")
                    |(testing_data_2['scheme_name'] == "Uroki-Bomang'ombe water sup"), 'scheme_name'] = "Uroki-Bomang'ombe water supply"

testing_data_2.loc[(testing_data_2['scheme_name'] == "wanging'ombe supply scheme")
                    |(testing_data_2['scheme_name'] == "wanging'ombe water supply s")
                    |(testing_data_2['scheme_name'] == "wanginyi water supplied sch"),'scheme_name'] = "wanging'ombe water supply scheme"
    
testing_data_2.loc[(testing_data_2['scheme_name'] == "World Bank Water Project")
                    |(testing_data_2['scheme_name'] == "World Bank Water Supply")
                    |(testing_data_2['scheme_name'] == "World banks")
                    |(testing_data_2['scheme_name'] == "World banks Water board")
                    |(testing_data_2['scheme_name'] == "World banks water supplying")
                    |(testing_data_2['scheme_name'] == "WOULD BANK"), 'scheme_name'] = "World Bank"                     
                      
testing_data_2.loc[(testing_data_2['scheme_name'] == "K")
                    |(testing_data_2['scheme_name'] == "M")
                    |(testing_data_2['scheme_name'] == "N")
                    |(testing_data_2['scheme_name'] == "S") 
                    |(testing_data_2['scheme_name'] == "D")
                    |(testing_data_2['scheme_name'] == "G")
                    |(testing_data_2['scheme_name'] == "J") 
                    |(testing_data_2['scheme_name'] == "U")
                    |(testing_data_2['scheme_name'] == "A")
                    |(testing_data_2['scheme_name'] == "W")
                    |(testing_data_2['scheme_name'] == "no scheme") 
                    |(testing_data_2['scheme_name'] == "B") 
                    |(testing_data_2['scheme_name'] == "I"), 'scheme_name']= np.nan

testing_data_2.loc[(testing_data_2['scheme_name'] == "Makwale water supplied sche"), 'scheme_name'] = "makwale water supplied scheme"
                     
#print(testing_data_2['scheme_name'].value_counts())


In [ ]:
#Converting wpt_name from None to nan in training
training_data_2.loc[(training_data_2['wpt_name'] == "none"), 'wpt_name']= np.nan
#print(training_data_2['wpt_name'].value_counts())

In [ ]:
#Converting wpt_name from None to nan in testing
testing_data_2.loc[(testing_data_2['wpt_name'] == "none"), 'wpt_name']= np.nan
#print(testing_data_2['wpt_name'].value_counts())

In [ ]:
#Converting some rows to missing in training data
training_data_2.loc[(training_data_2['subvillage'] == "M")
                    |(training_data_2['subvillage'] == "1")
                    |(training_data_2['subvillage'] == "I")
                    |(training_data_2['subvillage'] == "K") 
                    |(training_data_2['subvillage'] == "L")
                    |(training_data_2['subvillage'] == "U")
                    |(training_data_2['subvillage'] == "S") 
                    |(training_data_2['subvillage'] == "C"), 'subvillage']= np.nan
#print(training_data_2['subvillage'].value_counts())

In [ ]:
#Converting some rows to missing in testing data
testing_data_2.loc[(testing_data_2['subvillage'] == "M")
                    |(testing_data_2['subvillage'] == "1")
                    |(testing_data_2['subvillage'] == "I")
                    |(testing_data_2['subvillage'] == "K") 
                    |(testing_data_2['subvillage'] == "L")
                    |(testing_data_2['subvillage'] == "U")
                    |(testing_data_2['subvillage'] == "S") 
                    |(testing_data_2['subvillage'] == "C"), 'scheme_name']= np.nan
#print(testing_data_2['subvillage'].value_counts())

In [ ]:
#changing some row names in the extraction type feature in training data
 
training_data_2.loc[(training_data_2['extraction_type'] == "other - rope pump"), 'extraction_type'] = "rope pump"

training_data_2.loc[(training_data_2['extraction_type'] == "other - swn 81"), 'extraction_type'] = "swn 81"

training_data_2.loc[(training_data_2['extraction_type'] == "other - play pump"), 'extraction_type'] = "play pump"

training_data_2.loc[(training_data_2['extraction_type'] == "other - mkulima/shinyanga"), 'extraction_type'] = "other"
#print(training_data_2['extraction_type'].value_counts())

In [ ]:
#changing some row names in the extraction type feature in testing data

testing_data_2.loc[(testing_data_2['extraction_type'] == "other - rope pump"), 'extraction_type'] = "rope pump"

testing_data_2.loc[(testing_data_2['extraction_type'] == "other - swn 81"), 'extraction_type'] = "swn 81"

testing_data_2.loc[(testing_data_2['extraction_type'] == "other - play pump"), 'extraction_type'] = "play pump"

testing_data_2.loc[(testing_data_2['extraction_type'] == "other - mkulima/shinyanga"), 'extraction_type'] = "other"
#print(testing_data_2['extraction_type'].value_counts())

In [ ]:

#Chanigng the category name on other rope pump cateogry in the extraction type group feature
training_data_2.loc[(training_data_2['extraction_type_group'] == "other handpump"), 'extraction_type_group'] = "handpump"
training_data_2.loc[(training_data_2['extraction_type_group'] == "other motorpump"), 'extraction_type_group'] = "motorpump"
#changin the category name on the other swn 81 cateogry
#training_data_2.loc[(training_data_2['extraction_type'] == "other - swn 81"), 'extraction_type'] = "swn 81"

#changin the category name on the other play pump cateogry
#training_data_2.loc[(training_data_2['extraction_type'] == "other - play pump"), 'extraction_type'] = "play pump"

#changin the category name on the other mkulima/shinyanga cateogry
#training_data_2.loc[(training_data_2['extraction_type'] == "other - mkulima/shinyanga"), 'extraction_type'] = "mkulima/shinyanga"

#print(training_data_2['extraction_type'].value_counts())



In [ ]:
#Chanigng the category name on other rope pump cateogry 
testing_data_2.loc[(testing_data_2['extraction_type_group'] == "other handpump"), 'extraction_type_group'] = "handpump"
testing_data_2.loc[(testing_data_2['extraction_type_group'] == "other motorpump"), 'extraction_type_group'] = "motorpump"

Converts all instances in the dataset to be capitalized and consistent

In [ ]:
#Converting all instances to have consistent titling on training data

training_data_2 = training_data_2.applymap(lambda s:s.title() if type(s) == str else s)

#training_data_2.head()


In [ ]:
#Converting all instances to have consistent titling on testing data
testing_data_2 = testing_data_2.applymap(lambda s:s.title() if type(s) == str else s)

#testing_data_2.head()

In [ ]:
#Making further adjustments to new categories on training data

training_data_2.loc[(training_data_2['payment_type'] == "Unknown"), 'payment_type'] = np.nan
training_data_2.loc[(training_data_2['source_class'] == "Unknown"), 'source_class'] = np.nan
training_data_2.loc[(training_data_2['water_quality'] == "Unknown"), 'water_quality'] = np.nan
training_data_2.loc[(training_data_2['quantity'] == "Unknown"), 'quantity'] = np.nan
training_data_2.loc[(training_data_2['quantity_group'] == "Unknown"), 'quantity_group'] = np.nan
training_data_2.loc[(training_data_2['source'] == "Unknown"), 'source'] = np.nan
training_data_2.loc[(training_data_2['scheme_management'] == "None"), 'scheme_management'] = np.nan
training_data_2.loc[(training_data_2['management_group'] == "Unknown"), 'management_group'] = np.nan
training_data_2.loc[(training_data_2['management'] == "Unknown"), 'management'] = np.nan
training_data_2.loc[(training_data_2['payment'] == "Unknown"), 'payment'] = np.nan
training_data_2.loc[(training_data_2['source_class'] == "Unknown"), 'source_class'] = np.nan

training_data_2.isnull().sum()


In [ ]:
#Making further adjustments to new categories on testing data

testing_data_2.loc[(testing_data_2['payment_type'] == "Unknown"), 'payment_type'] = np.nan
testing_data_2.loc[(testing_data_2['source_class'] == "Unknown"), 'source_class'] = np.nan
testing_data_2.loc[(testing_data_2['water_quality'] == "Unknown"), 'water_quality'] = np.nan
testing_data_2.loc[(testing_data_2['quantity'] == "Unknown"), 'quantity'] = np.nan
testing_data_2.loc[(testing_data_2['quantity_group'] == "Unknown"), 'quantity_group'] = np.nan
testing_data_2.loc[(testing_data_2['source'] == "Unknown"), 'source'] = np.nan
testing_data_2.loc[(testing_data_2['scheme_management'] == "None"), 'scheme_management'] = np.nan
testing_data_2.loc[(testing_data_2['management_group'] == 'Unknown'), 'management_group'] = np.nan
testing_data_2.loc[(testing_data_2['management'] == "Unknown"), 'management'] = np.nan
testing_data_2.loc[(testing_data_2['payment'] == "Unknown"), 'payment'] = np.nan
testing_data_2.loc[(testing_data_2['source_class'] == "Unknown"), 'source_class'] = np.nan

testing_data_2.isnull().sum()

# Combining Rare Categories 

In [ ]:
#Creating rare categories for funder column 

def funder_wrangler(row):  
    
    if row['funder']=='Government Of Tanzania':
        return 'Government Of Tanzania'
    elif row['funder']=='Danida':
        return 'Danida'
    elif row['funder']=='Hesawa':
        return 'Hesawa'
    elif row['funder']=='Rwssp':
        return 'Rwssp'
    elif row['funder']=='World Bank':
        return 'World Bank'
    elif row['funder']=='Kkkt':
        return 'Kkkt'
    elif row['funder']=='World Vision':
        return 'World Vision'
    elif row['funder']=='Unicef':
        return 'Unicef'
    elif row['funder']=='Private Individual':
        return 'Private Individual'
    elif row['funder']=='Rc Church':
        return 'Rc Church'
    elif row['funder']=='Tasaf':
        return 'Tasaf'
    elif row['funder']=='Dwsp':
        return 'Dwsp'
    elif row['funder']=='District Council':
        return 'District Council'
    elif row['funder']=='Dhv':
        return 'Dhv'
    elif row['funder']=='0':
        return np.nan
    elif row['funder']=='Norad':
        return 'Norad'
    elif row['funder']=='Fini Water':
        return 'Fini Water'
    elif row['funder']=='Germany Republic':
        return 'Germany Republic'
    elif row['funder']=='Netherlands':
        return 'Netherlands'
    elif row['funder']=='Tcrs':
        return 'Tcrs'
    elif row['funder']=='Ministry Of Water':
        return 'Ministry Of Water'
    elif row['funder']=='Water':
        return 'Water'
    elif row['funder']=='Jaica':
        return 'Jaica'
    elif row['funder']=='Dwe':
        return 'Dwe'
    elif row['funder']=='Hifab':
        return 'Hifab'
    elif row['funder']=='Adb':
        return 'Adb'
    elif row['funder']=='Lga':
        return 'Lga'
    elif row['funder']=='Isf':
        return 'Isf'
    elif row['funder']=='Ces (gmbh)':
        return 'Ces (gmbh)'
    elif row['funder']=='Amref':
        return 'Amref'
    elif row['funder']=='Mission':
        return 'Mission'
    elif row['funder']=='Oxfam':
        return 'Oxfam'
    elif row['funder']=='Wateraid':
        return 'Wateraid'
    elif row['funder']=='Rudep':
        return 'Rudep'
    elif row['funder']=='Rural Water And Sanitation':
        return 'Rural Water And Sanitation'
    elif row['funder']=='Plan International':
        return 'Plan International'
    elif row['funder']=='Adra':
        return 'Adra'
    elif row['funder']=='Shipo':
        return 'Shipo'
    else:
        return 'other'

In [ ]:
#Calling funder wrangler fucntion on training data
training_data_2['funder'] = training_data_2.apply(lambda row: funder_wrangler(row), axis=1)

In [ ]:
#Calling funder wrangler fucntion on testing data
testing_data_2['funder'] = testing_data_2.apply(lambda row: funder_wrangler(row), axis=1)

In [ ]:
#Creating rare categories for installer column 
def installer_wrangler(row):
    if row['installer']=='Dwe':
        return 'Dwe'
    elif row['installer']=='Government':
        return 'Government'
    elif row['installer']=='Danida':
        return 'Danida'
    elif row['installer']=='Hesawa':
        return 'Hesawa'
    elif row['installer']=='Rwe':
        return 'Rwe'
    elif row['installer']=='KKKT':
        return 'KKKT'
    elif row['installer']=='Central Government':
        return 'Central Government'
    elif row['installer']=='District Council':
        return 'District Council'
    elif row['installer']=='Fini Water':
        return 'Fini Water'
    elif row['installer']=='0':
        return np.nan
    elif row['installer']=='World Vision':
        return 'World Vision'
    elif row['installer']=='Tcrs':
        return 'Tcrs'
    elif row['installer']=='Ces':
        return 'Ces'
    elif row['installer']=='Amref':
        return 'Amref'
    elif row['installer']=='Lga':
        return 'Lga'
    elif row['installer']=='Wedeco':
        return 'Wedeco'
    elif row['installer']=='Community':
        return 'Community'
    elif row['installer']=='Rc Church':
        return 'Rc Church'
    elif row['installer']=='Tasaf':
        return 'Tasaf'
    elif row['installer']=='Unicef':
        return 'Unicef'
    elif row['installer']=='Oxfam':
        return 'Oxfam'
    elif row['installer']=='Twesa':
        return 'Twesa'
    elif row['installer']=='Wu':
        return 'Wu'
    elif row['installer']=='Jaica':
        return 'Jaica'
    elif row['installer']=='Dmdd':
        return 'Dmdd'
    elif row['installer']=='Acra':
        return 'Acra'
    elif row['installer']=='Kiliwater':
        return 'Kiliwater'
    elif row['installer']=='World Vision':
        return 'World Vision'
    elif row['installer']=='Sema':
        return 'Sema'
    elif row['installer']=='Da':
        return 'Da'
    elif row['installer']=='Idara ya maji':
        return 'Idara ya maji'
    elif row['installer']=='Sengerema Water Department':
        return 'Sengerema Water Department'
    elif row['installer']=='Mission':
        return 'Mission'
    elif row['installer']=='Norad':
        return 'Norad'
    elif row['installer']=='Dh':
        return 'Dh'
    elif row['installer']=='Villagers':
        return 'Villagers'
    elif row['installer']=='Private':
        return 'Private'
    else:
        return 'other'

In [ ]:
#Calling installer wrangler fucntion on training data
training_data_2['installer'] = training_data_2.apply(lambda row: installer_wrangler(row), axis=1)

In [ ]:
#Calling installer wrangler fucntion on testing data
testing_data_2['installer'] = testing_data_2.apply(lambda row: installer_wrangler(row), axis=1)

In [ ]:
#Creating rare categories for scheme_name column 
def scheme_name_wrangler(row):
    if row['scheme_name']=='Borehole':
        return 'Borehole'
    elif row['scheme_name']=="Wanging'Ombe Water Supply Scheme":
        return "Wanging'Ombe Water Supply Scheme"
    elif row['scheme_name']=='Chalinze Water':
        return 'Chalinze Water'
    elif row['scheme_name']=='Danida':
        return 'Danida'
    elif row['scheme_name']=='Government':
        return 'Government'
    elif row['scheme_name']=='Ngana Water Supplied Scheme':
        return 'Ngana Water Supplied Scheme'
    elif row['scheme_name']=='Bagamoyo Water':
        return 'Bagamoyo Water'
    elif row['scheme_name']=='Lyamungo-Umbwe Water Supply':
        return 'Lyamungo-Umbwe Water Supply'
    elif row['scheme_name']=="Uroki-Bomang'Ombe Water Suppply":
        return "Uroki-Bomang'Ombe Water Suppply"
    elif row['scheme_name']=='Losaa-Kia Water Supply':
        return 'Losaa-Kia Water Supply'
    elif row['scheme_name']=='Kirua Kahe Gravity Water Supply Trust':
        return 'Kirua Kahe Gravity Water Supply Trust'
    elif row['scheme_name']=='Machumba Estate Pipe Line':
        return 'Machumba Estate Pipe Line'
    elif row['scheme_name']=='Makwale Water Supplied Scheme':
        return 'Makwale Water Supplied Scheme'
    elif row['scheme_name']=='Kijiji':
        return 'Kijiji'
    elif row['scheme_name']=='Mkongoro One':
        return 'Mkongoro One'
    elif row['scheme_name']=='Handeni Trunk Maini':
        return 'Handeni Trunk Maini'
    elif row['scheme_name']=='Mtwango Water Supply Scheme':
        return 'Mtwango Water Supply Scheme'
    elif row['scheme_name']=='Rc Church':
        return 'Rc Church'
    elif row['scheme_name']=='Mkongoro Two':
        return 'Mkongoro Two'
    elif row['scheme_name']=='Upper Ruvu':
        return 'Upper Ruvu'
    elif row['scheme_name']=='Tove-Mtwango Water Supply Scheme':
        return 'Tove-Mtwango Water Supply Scheme'
    elif row['scheme_name']=='Imalinyi Water Supply Scheme':
        return 'Imalinyi Water Supply Scheme'
    elif row['scheme_name']=='Maambreni Gravity Water Supply':
        return 'Maambreni Gravity Water Supply'
    elif row['scheme_name']=='Shallow Well':
        return 'Shallow Well'
    elif row['scheme_name']=='Ngamanga Water Supplied Scheme':
        return 'Ngamanga Water Supplied Scheme'
    elif row['scheme_name']=='Kirua Kahe Pumping Water Trust':
        return 'Kirua Kahe Pumping Water Trust'
    elif row['scheme_name']=='World Bank':
        return 'World Bank'
    elif row['scheme_name']=='Olkokola-Mwandet Pipeline':
        return 'Olkokola-Mwandet Pipeline'
    elif row['scheme_name']=='Ki':
        return 'Ki'
    elif row['scheme_name']=='Nabaiye Pipe Line':
        return 'Nabaiye Pipe Line'
    elif row['scheme_name']=='Nasula Gravity Water Supply':
        return 'Nasula Gravity Water Supply'
    elif row['scheme_name']=='Sinyanga Water Supply Scheme':
        return 'Sinyanga Water Supply Scheme'
    else:
        return 'other'


In [ ]:
#Calling scheme_name wrangler fucntion on training data
training_data_2['scheme_name'] = training_data_2.apply(lambda row: scheme_name_wrangler(row), axis=1)

In [ ]:
#Calling scheme_name wrangler fucntion on testing data
testing_data_2['scheme_name'] = testing_data_2.apply(lambda row: scheme_name_wrangler(row), axis=1)

In [ ]:
#Creating rare categories for wpt_name column 
def wpt_name_wrangler(row):
    if row['wpt_name']=='Shuleni':
        return 'Shuleni'
    elif row['wpt_name']=="Zahanati":
        return "Zahanati"
    elif row['wpt_name']=='Msikitini':
        return 'Msikitini'
    elif row['wpt_name']=='Kanisani':
        return 'Kanisani'
    elif row['wpt_name']=='Bombani':
        return 'Bombani'
    elif row['wpt_name']=='Sokoni':
        return 'Sokoni'
    elif row['wpt_name']=='Ofisini':
        return 'Ofisini'
    elif row['wpt_name']=='School':
        return 'School'
    elif row['wpt_name']=="Shule Ya Msingi":
        return "Shule Ya Msingi"
    elif row['wpt_name']=='Shule':
        return 'Shule'
    elif row['wpt_name']=='Sekondari':
        return 'Sekondari'
    elif row['wpt_name']=='Muungano':
        return 'Muungano'
    elif row['wpt_name']=='Mkombozi':
        return 'Mkombozi'
    elif row['wpt_name']=='Madukani':
        return 'Madukani'
    else:
        return 'other'


In [ ]:
#Calling wpt_name wrangler fucntion on training data
training_data_2['wpt_name'] = training_data_2.apply(lambda row: wpt_name_wrangler(row), axis=1)

In [ ]:
#Calling wpt_name wrangler fucntion on testing data
testing_data_2['wpt_name'] = testing_data_2.apply(lambda row: wpt_name_wrangler(row), axis=1)

In [ ]:
#Creating rare categories for subvillage column 
def subvillage_wrangler(row):
    if row['subvillage']=='Madukani':
        return 'Madukani'
    elif row['subvillage']=="Shuleni":
        return "Shuleni"
    elif row['subvillage']=='Majengo':
        return 'Majengo'
    elif row['subvillage']=='Kati':
        return 'Kati'
    elif row['subvillage']=='Mtakuja':
        return 'Mtakuja'
    elif row['subvillage']=='Sokoni':
        return 'Sokoni'
    elif row['subvillage']=='Muungano':
        return 'Muungano'
    elif row['subvillage']=='Mbuyuni':
        return 'Mbuyuni'
    elif row['subvillage']=="Mlimani":
        return "Mlimani"
    elif row['subvillage']=='Songambele':
        return 'Songambele'
    elif row['subvillage']=='Miembeni':
        return 'Miembeni'
    elif row['subvillage']=='Msikitini':
        return 'Msikitini'
    elif row['subvillage']=='Kibaoni':
        return 'Kibaoni'
    elif row['subvillage']=='Kanisani':
        return 'Kanisani'
    elif row['subvillage']=='Mapinduzi':
        return 'Mapinduzi'
    elif row['subvillage']=='Mjini':
        return 'Mjini'
    elif row['subvillage']=='Mjimwema':
        return 'Mjimwema'
    elif row['subvillage']=='Mkwajuni':
        return 'Mkwajuni'
    elif row['subvillage']=='Mwenge':
        return 'Mwenge'
    else:
        return 'other'


In [ ]:
#Calling subvillage wrangler fucntion on training data
training_data_2['subvillage'] = training_data_2.apply(lambda row: subvillage_wrangler(row), axis=1)

In [ ]:
#Calling subvillage wrangler fucntion on testing data
testing_data_2['subvillage'] = testing_data_2.apply(lambda row: subvillage_wrangler(row), axis=1)

In [ ]:
#Creating rare categories for lga column 
def lga_wrangler(row):
    if row['lga']=='Njombe':
        return 'Njombe'
    elif row['lga']=="Arusha Rural":
        return "Arusha Rural"
    elif row['lga']=='Moshi Rural':
        return 'Moshi Rural'
    elif row['lga']=='Bariadi':
        return 'Bariadi'
    elif row['lga']=='Rungwe':
        return 'Rungwe'
    elif row['lga']=='Kilosa':
        return 'Kilosa'
    elif row['lga']=='Kasulu':
        return 'Kasulu'
    elif row['lga']=='Mbozi':
        return 'Mbozi'
    elif row['lga']=="Meru":
        return "Meru"
    elif row['lga']=='Bagamoyo':
        return 'Bagamoyo'
    elif row['lga']=='Singida Rural':
        return 'Singida Rural'
    elif row['lga']=='Kilombero':
        return 'Kilombero'
    elif row['lga']=='Same':
        return 'Same'
    elif row['lga']=='Kibondo':
        return 'Kibondo'
    elif row['lga']=='Kyela':
        return 'Kyela'
    elif row['lga']=='Kahama':
        return 'Kahama'
    elif row['lga']=='Magu':
        return 'Magu'
    elif row['lga']=='Kigoma Rural':
        return 'Kigoma Rural'
    elif row['lga']=='Maswa':
        return 'Maswa'
    elif row['lga']=='Karagwe':
        return 'Karagwe'
    elif row['lga']=='Mbinga':
        return 'Mbinga'
    elif row['lga']=='Iringa Rural':
        return 'Iringa Rural'
    elif row['lga']=='Serengeti':
        return 'Serengeti'
    elif row['lga']=='Namtumbo':
        return 'Namtumbo'
    elif row['lga']=='Lushoto':
        return 'Lushoto'
    elif row['lga']=='Songea Rural':
        return 'Songea Rural'
    elif row['lga']=='Mpanda':
        return 'Mpanda'
    elif row['lga']=='Mvomero':
        return 'Mvomero'
    elif row['lga']=='Ngara':
        return 'Ngara'
    elif row['lga']=='Ulanga':
        return 'Ulanga'
    elif row['lga']=='Makete':
        return 'Makete'
    elif row['lga']=='Kwimba':
        return 'Kwimba'
    elif row['lga']=='Mbarali':
        return 'Mbarali'
    elif row['lga']=='Hai':
        return 'Hai'
    elif row['lga']=='Rombo':
        return 'Rombo'
    elif row['lga']=='Shinyanga Rural':
        return 'Shinyanga Rural'
    elif row['lga']=='Nzega':
        return 'Nzega'
    elif row['lga']=='Ludewa':
        return 'Ludewa'
    elif row['lga']=='Mkuranga':
        return 'Mkuranga'
    elif row['lga']=='Iramba':
        return 'Iramba'
    elif row['lga']=='Masasi':
        return 'Masasi'
    elif row['lga']=='Kondoa':
        return 'Kondoa'
    elif row['lga']=='Morogoro Rural':
        return 'Morogoro Rural'
    elif row['lga']=='Sumbawanga Rural':
        return 'Sumbawanga Rural'
    elif row['lga']=='Mufindi':
        return 'Mufindi'
    elif row['lga']=='Mwanga':
        return 'Mwanga'
    elif row['lga']=='Bukombe':
        return 'Bukombe'
    elif row['lga']=='Babati':
        return 'Babati'
    else:
        return 'other'


In [ ]:
#Calling lga wrangler fucntion on training data
training_data_2['lga'] = training_data_2.apply(lambda row: lga_wrangler(row), axis=1)

In [ ]:
#Calling subvillage wrangler fucntion on testing data
testing_data_2['lga'] = testing_data_2.apply(lambda row: lga_wrangler(row), axis=1)

In [ ]:
#Creating rare categories for ward column 
def ward_wrangler(row):
    if row['ward']=='Igosi':
        return 'Igosi'
    elif row['ward']=="Imalinyil":
        return "Imalinyi"
    elif row['ward']=='Siha Kati':
        return 'Siha Kati'
    elif row['ward']=='Mdandu':
        return 'Mdandu'
    elif row['ward']=='Nduruma':
        return 'Nduruma'
    elif row['ward']=='Mishamo':
        return 'Mishamo'
    elif row['ward']=='Kitunda':
        return 'Kitunda'
    elif row['ward']=='Msindo':
        return 'Msindo'
    elif row['ward']=="Chalinze":
        return "Chalinze"
    elif row['ward']=='Maji Ya Chai':
        return 'Maji Ya Chai'
    elif row['ward']=='Usuka':
        return 'Usuka'
    elif row['ward']=='Ngarenanyuki':
        return 'Ngarenanyuki'
    elif row['ward']=='Chanika':
        return 'Chanika'
    elif row['ward']=='Vikindu':
        return 'Vikindu'
    elif row['ward']=='Matola':
        return 'Matola'
    elif row['ward']=='Zinga/Ikerege':
        return 'Zinga/Ikerege'
    elif row['ward']=="Wanging'Ombe":
        return "Wanging'Ombe"
    elif row['ward']=='Maramba':
        return 'Maramba'
    elif row['ward']=='Itete':
        return 'Itete'
    elif row['ward']=='Magomeni':
        return 'Magomeni'
    elif row['ward']=='Kikatiti':
        return 'Kikatiti'
    elif row['ward']=='Ifakara':
        return 'Ifakara'
    elif row['ward']=='Olkokola':
        return 'Olkokola'
    elif row['ward']=='Maposeni':
        return 'Maposeni'
    elif row['ward']=='Igongolo':
        return 'Igongolo'
    elif row['ward']=='Mvomero':
        return 'Mvomero'
    elif row['ward']=='Mpanda':
        return 'Mpanda'
    elif row['ward']=='Mvomero':
        return 'Mvomero'
    elif row['ward']=='Mlangali':
        return 'Mlangali'
    elif row['ward']=='Nkoma':
        return 'Nkoma'
    elif row['ward']=='Mahongole':
        return 'Mahongole'
    elif row['ward']=='Nkungulu':
        return 'Nkungulu'
    elif row['ward']=='Rujewa':
        return 'Rujewa'
    elif row['ward']=='Simbo':
        return 'Simbo'
    elif row['ward']=='Masama Magharibi':
        return 'Masama Magharibi'
    elif row['ward']=='Kiranyi':
        return 'Kiranyi'
    elif row['ward']=='Mamire':
        return 'Mamire'
    elif row['ward']=='Lupalilo':
        return 'Lupalilo'
    elif row['ward']=='Kidatu':
        return 'Kidatu'
    elif row['ward']=='Kagongo':
        return 'Kagongo'
    elif row['ward']=='Ihanda':
        return 'Ihanda'
    elif row['ward']=='Hedaru':
        return 'Hedaru'
    elif row['ward']=='Chinamili':
        return 'Chinamili'
    elif row['ward']=='Isongole':
        return 'Isongole'
    elif row['ward']=='Siha Mashariki':
        return 'Siha Mashariki'
    elif row['ward']=='Mkongo':
        return 'Mkongo'
    elif row['ward']=='Masama Mashariki':
        return 'Masama Mashariki'
    elif row['ward']=='Tinde':
        return 'Tinde'
    elif row['ward']=='Makuyuni':
        return 'Makuyuni'
    elif row['ward']=='Yombo':
        return 'Yombo'
    elif row['ward']=='Kanga':
        return 'Kanga'
    else:
        return 'other'


In [ ]:
#Calling ward wrangler fucntion on training data
training_data_2['ward'] = training_data_2.apply(lambda row: ward_wrangler(row), axis=1)

In [ ]:
#Calling subvillage wrangler fucntion on testing data
testing_data_2['ward'] = testing_data_2.apply(lambda row: ward_wrangler(row), axis=1)

In [ ]:
# Turn construction_year into bins

def construction_wrangler(row):
    if row['construction_year'] >= 1960 and row['construction_year'] < 1970:
        return '60s'
    elif row['construction_year'] >= 1970 and row['construction_year'] < 1980:
        return '70s'
    elif row['construction_year'] >= 1980 and row['construction_year'] < 1990:
        return '80s'
    elif row['construction_year'] >= 1990 and row['construction_year'] < 2000:
        return '90s'
    elif row['construction_year'] >= 2000 and row['construction_year'] < 2010:
        return '00s'
    elif row['construction_year'] >= 2010:
        return '10s'
    else:
        return np.nan
    


In [ ]:
#Calling construction wrangler for training dataset
training_data_2['construction_year'] = training_data_2.apply(lambda row: construction_wrangler(row), axis=1)

In [ ]:
#Calling construction wrangler for testing dataset
testing_data_2['construction_year'] = testing_data_2.apply(lambda row: construction_wrangler(row), axis=1)

In [ ]:
#converting district and region code feature to object datatype in training file
training_data_2['district_code'] = training_data_2['district_code'].astype('object')
training_data_2['region_code'] = training_data_2['region_code'].astype('object')
training_data_2['construction_year'] = training_data_2['construction_year'].astype('object')


In [ ]:
training_data_2.info()

In [ ]:
#converting district and region code feature to object datatype in testing file
testing_data_2['district_code'] = testing_data_2['district_code'].astype('object')
testing_data_2['region_code'] = testing_data_2['region_code'].astype('object')
testing_data_2['construction_year'] = testing_data_2['construction_year'].astype('object')

In [ ]:
#Normalizing population and gps height in training set
training_data_2['population']=np.log10(training_data_2['population']+1)
training_data_2['gps_height']=np.log10(training_data_2['gps_height']+100)

In [ ]:
#Normalizing population and gps height in testing set
testing_data_2['population']=np.log10(testing_data_2['population']+1)
testing_data_2['gps_height']=np.log10(testing_data_2['gps_height']+100)

# Dropping columns

In [ ]:
#Dropping features in training data that we don't need

training_data_2=training_data_2.drop(['id','recorded_by','num_private', 'date_recorded'], axis='columns')


#Dropping features in testing data that we don't need
testing_data_2=testing_data_2.drop(['id','recorded_by','num_private', 'date_recorded'], axis='columns')
training_data_2.shape
testing_data_2.shape


# Fixing Missing Values

In [ ]:
#Imputing Missing Values for TRAINING set 
# Approach:
# Categorical features: adding new category 'missing_value' (as if this is a new color, or gender)
# Numerical features: replacing with a median (or mean, or most frequent value, etc.) An alternative approach is to imputate, see here: https://scikit-learn.org/stable/modules/impute.html 
# + we add surrogate columns indicating that the value is imputed

# creating surrogates
#for col in training_data_2:
#    if training_data_2[col].isna().sum() != 0: 
#        training_data_2[col + '_surrogate'] = training_data_2[col].isna().astype(int)

# fixing categoricals
imputer = SimpleImputer(missing_values = np.nan, strategy='constant')
imputer.fit(training_data_2.select_dtypes(exclude=['int64','float64']))
training_data_2[training_data_2.select_dtypes(exclude=['int64','float64']).columns] = imputer.transform(training_data_2.select_dtypes(exclude=['int64','float64']))
           
# fixing numericals 
#imputer = SimpleImputer(missing_values = np.nan, strategy='median')
#imputer.fit(training_data_2.select_dtypes(include=['int64','float64']))
#training_data_2[training_data_2.select_dtypes(include=['int64','float64']).columns] = imputer.transform(training_data_2.select_dtypes(include=['int64','float64']))

training_data_2.head(150)


In [ ]:
#Imputing Missing Values for TESTING set 
# Approach:
# Categorical features: adding new category 'missing_value' (as if this is a new color, or gender)
# Numerical features: replacing with a median (or mean, or most frequent value, etc.) An alternative approach is to imputate, see here: https://scikit-learn.org/stable/modules/impute.html 
# + we add surrogate columns indicating that the value is imputed

# creating surrogates
#for col in testing_data_2:
#    if testing_data_2[col].isna().sum() != 0: 
#        testing_data_2[col + '_surrogate'] = testing_data_2[col].isna().astype(int)

# fixing categoricals
imputer = SimpleImputer(missing_values = np.nan, strategy='constant')
imputer.fit(testing_data_2.select_dtypes(exclude=['int64','float64']))
testing_data_2[testing_data_2.select_dtypes(exclude=['int64','float64']).columns] = imputer.transform(testing_data_2.select_dtypes(exclude=['int64','float64']))
           
# fixing numericals 
#imputer = SimpleImputer(missing_values = np.nan, strategy='median')
#imputer.fit(training_data_2.select_dtypes(include=['int64','float64']))
#training_data_2[training_data_2.select_dtypes(include=['int64','float64']).columns] = imputer.transform(training_data_2.select_dtypes(include=['int64','float64']))

testing_data_2.head(150)

In [ ]:
#Checking the shape of both the training and testing columsn 
training_data_2.shape
testing_data_2.shape

# One Hot Encoding

In [ ]:
training_data_2.info()

In [ ]:
#creating dummy variables for non-numbers ("one hot encoding") on TRAINING Set 
training_data_3 = training_data_2.copy()
training_data_3 = pd.get_dummies(training_data_3, columns = training_data_3.select_dtypes(exclude=['int64','float64']).columns, drop_first = True)

training_data_3.head()
training_data_3.shape
training_data_3.info()



In [ ]:
#creating dummy variables for non-numbers ("one hot encoding") on TESTING
testing_data_3 = testing_data_2.copy()
testing_data_3 = pd.get_dummies(testing_data_3, columns = testing_data_3.select_dtypes(exclude=['int64','float64']).columns, drop_first = True)

testing_data_3.head()
testing_data_3.shape
testing_data_3.info()

# Label Encoding 

In [ ]:
#label encoding for models in training set 
cat_df=['quantity','management_group','management','extraction_type_class','extraction_type_group',
       'extraction_type','permit','scheme_name','scheme_management','public_meeting','ward',
       'lga','region','subvillage','basin','waterpoint_type_group','waterpoint_type','source_class',
       'source_type','source','quality_group','water_quality','payment_type','payment','wpt_name','installer'
       ,'funder','quantity_group']

for n in cat_df:
    
    training_data_2[n]= training_data_2[n].astype('category').cat.codes

In [ ]:
cat_df_test=['quantity','management_group','management','extraction_type_class','extraction_type_group',
       'extraction_type','permit','scheme_name','scheme_management','public_meeting','ward',
       'lga','region','subvillage','basin','waterpoint_type_group','waterpoint_type','source_class',
       'source_type','source','quality_group','water_quality','payment_type','payment','wpt_name','installer'
       ,'funder','quantity_group']


for n in cat_df_test:
    
    testing_data_2[n]= testing_data_2[n].astype('category').cat.codes

In [ ]:
training_data_2.shape
testing_data_2.shape
training_data_2.head()

# Models

In [ ]:
#Creating test and train sets
from sklearn.model_selection import train_test_split
X=training_data_2.drop(['Status_group'],axis=1)
y=training_data_2[['Status_group']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

type(X_train)
type(y_train)

Random Forest Model version 1 - Best Model

In [ ]:
#Random Forest

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
training_data_2['Status_group'] = training_data_2['Status_group'].astype('int64')

clf_rf = RandomForestClassifier(n_estimators=600,max_depth=22,random_state=0,class_weight='balanced')

clf_rf.fit(X_train, y_train.values.ravel())

In [ ]:
y_pred_rf = clf_rf.predict(X_test)

In [ ]:
#Generating the confusion Matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred_rf)

In [ ]:
#Printing out the classification report for Random Forest Algorithm
from sklearn.metrics import classification_report
class_names = [str(x) for x in clf_rf.classes_]
print(classification_report(y_test,y_pred_rf,target_names=class_names))

In [ ]:
#pd.DataFrame(y_pred).to_csv("rf_submission.csv")

Random Forest Model Version 2

In [ ]:
clf_rf_2 = RandomForestClassifier(criterion='entropy',
                                min_samples_split=10,
                                n_estimators=700,
                                max_features='auto',
                                oob_score=True,
                                random_state=0,
                                class_weight = 'balanced',
                                n_jobs=-1)

clf_rf_2.fit(X_train, y_train.values.ravel())

In [ ]:
y_pred_rf_2 = clf_rf_2.predict(X_test)

In [ ]:
#Generating the confusion Matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred_rf_2)

In [ ]:
#Printing out the classification report for Random Forest Algorithm
from sklearn.metrics import classification_report
class_names = [str(x) for x in clf_rf_2.classes_]
print(classification_report(y_test,y_pred_rf_2,target_names=class_names))

In [ ]:
prediction_submission = clf_rf_2.predict(testing_data_2)

In [ ]:
pd.DataFrame(prediction_submission).to_csv("rf_2_submission.csv")

Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

clf = DecisionTreeClassifier(random_state=42)

params = {"criterion": ["gini", "entropy"],
          "splitter": ["best", "random"],
          "class_weight": ['balanced', None], 
          "max_depth": randint(2, 21),
          "min_samples_leaf": randint(1, 11),
          "max_features": uniform(0.0, 1.0)}

search = RandomizedSearchCV(clf, param_distributions=params, n_iter=1000, scoring='f1_macro', cv=10, verbose=1)
search = search.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report
y_pred = search.predict(X_cv)
print(classification_report(y_cv, y_pred))

Gradient Boosting Classifier

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

def model(X_train, X_val, y_train, y_val, test):
    if __name__ == '__main__':
    
        param_grid = {'learning_rate': [0.25,0.5,0.75],
                      'max_depth': [14,18,22],
                      'min_samples_leaf': [16,20],
                      'max_features': [1.0],
                      'n_estimators': [500]}                      

        estimator = GridSearchCV(estimator=GradientBoostingClassifier(),
                                 param_grid=param_grid,
                                 n_jobs=-1)

        estimator.fit(X_train, y_train)

        best_params = estimator.best_params_

        print (best_params)
                                 
        validation_accuracy = estimator.score(X_val, y_val)
        print('Validation accuracy: ', validation_accuracy)

%time model(X_train, X_cv, Y_train, y_cv, test)

In [ ]:
model(X_train, X_cv, Y_train, y_cv, test)